# Dataset for the Parquet files

In [101]:
from parquet_dataset.parquet_dataset import ParquetDataset
from parquet_dataset.parquet_dataset import DatetimeParquetDataset
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Instantiate the dataloader.

In [102]:
from typing import List

def make_train_parquet_path(i: int) -> str:
    return f"/kaggle/input/jane-street-real-time-market-data-forecasting/train.parquet/partition_id={i}/part-0.parquet"
# Setup the file indices to use.
K_MAX_TRAIN_FILES: int = 10
K_TRAIN_FILES: List[str] = [make_train_parquet_path(i) for i in range(K_MAX_TRAIN_FILES)]
K_TEST_FILES: List[str] = ["/kaggle/input/jane-street-real-time-market-data-forecasting/test.parquet/date_id=0/part-0.parquet"]

train_dataset = ParquetDataset(file_paths=K_TRAIN_FILES, logging=True)

Loaded files with rows:
	1944210 : /kaggle/input/jane-street-real-time-market-data-forecasting/train.parquet/partition_id=0/part-0.parquet
	2804247 : /kaggle/input/jane-street-real-time-market-data-forecasting/train.parquet/partition_id=1/part-0.parquet
	3036873 : /kaggle/input/jane-street-real-time-market-data-forecasting/train.parquet/partition_id=2/part-0.parquet
	4016784 : /kaggle/input/jane-street-real-time-market-data-forecasting/train.parquet/partition_id=3/part-0.parquet
	5022952 : /kaggle/input/jane-street-real-time-market-data-forecasting/train.parquet/partition_id=4/part-0.parquet
	5348200 : /kaggle/input/jane-street-real-time-market-data-forecasting/train.parquet/partition_id=5/part-0.parquet
	6203912 : /kaggle/input/jane-street-real-time-market-data-forecasting/train.parquet/partition_id=6/part-0.parquet
	6335560 : /kaggle/input/jane-street-real-time-market-data-forecasting/train.parquet/partition_id=7/part-0.parquet
	6140024 : /kaggle/input/jane-street-real-time-market-da

In [ ]:
dt_train_dataset = DatetimeParquetDataset(
    file_paths=K_TRAIN_FILES[0:2], 
    logging=False,
    time_context_length=3
)

In [106]:
print(len(dt_train_dataset))
for i in range(len(dt_train_dataset)):
    if i % 10000 == 0:
        print(i)
    dt_train_dataset[i]
    # print(dt_train_dataset.date_time_segments[i])
    # print(dt_train_dataset[i].size())


288660
0


RuntimeError: stack expects each tensor to be equal size, but got [8, 92] at entry 0 and [12, 92] at entry 2

In [ ]:
# Load a sample to populate the row group cache
idx = 0
symbols = set()
dates = set()
while idx < len(train_dataset):
    train_dataset[idx]
    idx += len(train_dataset.pq_cache.df)
    print(train_dataset.pq_cache.df.head())
    print(train_dataset.pq_cache.df.tail())
    symbols.update(set(train_dataset.pq_cache.df.get_column("symbol_id")))
    dates.update(set(train_dataset.pq_cache.df.get_column("date_id")))

print(symbols)
print(dates)

# Create a custom sample for randomly ordering the parquet files and rows within the parquet files

In [22]:
from parquet_sampler.parquet_sampler import ParquetBatchedSampler
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Create the DataLoader

In [23]:
dl = DataLoader(
    train_dataset, 
    batch_size=64,
    num_workers=12,
    prefetch_factor=2,
    sampler=ParquetBatchedSampler(data_source=train_dataset)
)


# Prepare data

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
# Separate features and responders
features = sample_df.filter(regex='^feature_')
responders = sample_df.filter(regex='^responder_')
# Convert to numpy arrays for TensorFlow
X = features.values  # Features for input
y = responders.values  # Responders for output
X = np.nan_to_num(X, nan=0.0, posinf=0.0, neginf=0.0)
y = np.nan_to_num(y, nan=0.0, posinf=0.0, neginf=0.0)

# Build the Autoencoder Model

In [ ]:
# Define the number of input and output nodes
input_dim = X.shape[1]  # Number of features (79)
output_dim = y.shape[1]  # Number of responders (9)
# Define the model
model = models.Sequential([
    layers.Input(shape=(input_dim,)),  # Input layer
    layers.Dense(64, activation='relu'),  # Encoder
    layers.Dense(32, activation='relu'),  # Bottleneck layer (compression)
    layers.Dense(64, activation='relu'),  # Decoder
    layers.Dense(output_dim, activation='linear')  # Output layer for responders
])
model.compile(optimizer='adam', loss='mse')

# Train Autoencoder Model

In [ ]:
from tensorflow.keras.callbacks import LearningRateScheduler
def step_decay(epoch):
    initial_lr = 0.01
    drop = 0.5
    epochs_drop = 5
    lr = initial_lr * (drop ** (epoch // epochs_drop))
    return lr
lr_scheduler = LearningRateScheduler(step_decay)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
# Define EarlyStopping
early_stopping = EarlyStopping(
    monitor='val_loss',    # Monitor validation loss
    patience=10,            # Number of epochs to wait for improvement
    min_delta=0.001,       # Minimum change to qualify as an improvement
    restore_best_weights=True  # Restore weights from the best epoch
)

history = model.fit(
    X, y,
    epochs=50,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stopping, lr_scheduler]
)

In [ ]:
model.save("/kaggle/working/model.keras")

# Submission

See [Jane Street RMF Demo Submission](https://www.kaggle.com/code/ryanholbrook/jane-street-rmf-demo-submission) for details

In [ ]:
import os
import polars as pl
import kaggle_evaluation.jane_street_inference_server

In [ ]:
import polars as pl
import numpy as np
# Assuming `model` is your trained model
# Assuming features required by the model are named 'feature_00', 'feature_01', etc.
def predict(test: pl.DataFrame, lags: pl.DataFrame | None) -> pl.DataFrame | pd.DataFrame:
    """Make a prediction."""
    global lags_
    if lags is not None:
        lags_ = lags
    # Extract the features for the model input
    feature_columns = [col for col in test.columns if col.startswith("feature_")]
    features = test.select(feature_columns).to_numpy()  # Convert to numpy array for model input
    features = np.nan_to_num(features, nan=0.0, posinf=0.0, neginf=0.0)
    # Generate predictions using the model
    model_predictions = model.predict(features)
    responder_6_predictions = model_predictions[:, 6]  # Assuming responder_6 is at index 6
    # Create a new Polars DataFrame with row_id and responder_6 predictions
    predictions = test.select("row_id").with_columns(
        pl.Series("responder_6", responder_6_predictions)
    )
    # Ensure the output format and length requirements
    if isinstance(predictions, pl.DataFrame):
        assert predictions.columns == ['row_id', 'responder_6']
    elif isinstance(predictions, pd.DataFrame):
        assert (predictions.columns == ['row_id', 'responder_6']).all()
    else:
        raise TypeError('The predict function must return a DataFrame')
    
    assert len(predictions) == len(test)
    return predictions

In [ ]:
inference_server = kaggle_evaluation.jane_street_inference_server.JSInferenceServer(predict)
if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(
        (
            '/kaggle/input/jane-street-real-time-market-data-forecasting/test.parquet',
            '/kaggle/input/jane-street-real-time-market-data-forecasting/lags.parquet',
        )
    )